# 1.1- Creación de una API con Flask

**[Documentación](https://flask.palletsprojects.com/en/1.1.x/)**

$$$$

![flask_api](images/flask_api.webp)

$$$$

Flask es un Microframework de Python que está basado en Werkzeug, Jinja 2 y buenas intenciones. Mediante Flask podemos construir aplicaciones web y servicios Restful (APIs) con Python de una forma extraordinariamente sencilla. Con pocas líneas podemos llegar a tener un servicio Restful funcionando. La mayor virtud de Flask es poder crear rutas web de una forma muy sencilla. 

In [ ]:
!pip install flask

### Hola mundo en Flask

In [2]:
from flask import Flask  # se importa la libreria


app=Flask(__name__)  # se inicia la aplicacion de Flask


@app.route('/')            # el decorador añade la funcion a la app de flask y le da el endpoint '/'
def saludar():             # la funcion devuelve una string
    return '¡hola Mundo!'


if __name__=='__main__':   # cuando se ejecute este archivo..
    app.run(debug=False)   # inicia la app, sin debug para jupyter

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


http://127.0.0.1:5000/

### Usando varios endpoints y parámetros

In [3]:
from flask import Flask, request


app=Flask(__name__)  


@app.route('/')            
def main():             
    return '¡Hola Mundo! Esto es una calculadora. Solo Suma y Resta.'


@app.route('/sumar/')
def sumar():
    
    n1=float(request.args.get('n1'))
    n2=float(request.args.get('n2'))
    
    return f'El resultado de sumar {n1} y {n2} es {n1+n2}'


@app.route('/restar/')
def restar():
    
    n1=float(request.args.get('n1'))
    n2=float(request.args.get('n2'))
    
    return f'El resultado de restar {n1} y {n2} es {n1-n2}'
    
    
if __name__=='__main__':  
    app.run(debug=False)   

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


2022-02-28 09:30:39,535 [23468] ERROR    __main__:1457: [JupyterRequire] Exception on /sumar/ [GET]
Traceback (most recent call last):
  File "/Users/iudh/opt/anaconda3/envs/clase/lib/python3.8/site-packages/flask/app.py", line 2073, in wsgi_app
    response = self.full_dispatch_request()
  File "/Users/iudh/opt/anaconda3/envs/clase/lib/python3.8/site-packages/flask/app.py", line 1518, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/Users/iudh/opt/anaconda3/envs/clase/lib/python3.8/site-packages/flask/app.py", line 1516, in full_dispatch_request
    rv = self.dispatch_request()
  File "/Users/iudh/opt/anaconda3/envs/clase/lib/python3.8/site-packages/flask/app.py", line 1502, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**req.view_args)
  File "<ipython-input-3-f322501fd7e4>", line 16, in sumar
    n2=float(request.args.get('n2'))
TypeError: float() argument must be a string or a number, not 'NoneType'


+ Página principal: http://127.0.0.1:5000/
+ Sumar: http://127.0.0.1:5000/sumar?n1=3&n2=20
+ Restar: http://127.0.0.1:5000/restar?n1=3&n2=20

### Creando una API 

In [4]:
from flask import Flask, request, render_template

import pandas as pd


app=Flask(__name__)  


data=None  # inicializo variables globales


@app.before_first_request  # antes de la primera llamada a la api, haz esto
def startup():
    
    global data
    
    data=pd.read_csv('data/apple_store.csv')

    
    
@app.route('/')            
def main():             
    return 'Apple Store API'


    
@app.route('/calc/')            
def calcular(): 
    # esta funcion devuelve la descripcion del dataframe
    
    return str(data.describe().T.to_json())



@app.route('/filtrar/')
def filtrar():
    # funcion para filtar por genero
    
    param=request.args.get('gen')
    
    filtrado=data[data.prime_genre==param].to_json()
    
    return str(filtrado)



@app.route('/selec/')  # ruta de la url
def seleccionar():
    # funcion que selecciona parte del dataframe
    
    # parametros, columnas a seleccionar
    params=request.args   # diccionario, key=nombre del parámetro, value=valor aportado en la url
    
    cols=[v for v in params.values()]  # valores, en este caso nombres de columna
   
    seleccion=data[cols].to_json()
    
    return str(seleccion)


@app.route('/mostrar/')
def mostrar():
    
    return render_template('base.html', tables=[data.to_html(index=False, render_links=True)])




if __name__=='__main__':  
    app.run(debug=False)   

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


+ Página principal: http://127.0.0.1:5000/

+ Cálculo de la descripción: http://127.0.0.1:5000/calc/

+ Filtrado por género: http://127.0.0.1:5000/filtrar/?gen=Games

+ Selección de columnas: http://127.0.0.1:5000/selec/?c1=user_rating&c2=user_rating_ver

+ Mostrar dataframe: http://127.0.0.1:5000/mostrar/

### Haciendo el despliegue (deploy)


PythonAnywhere es un servicio para ejecutar código python en servidores 'en la nube'. Lo vamos a usar para alojar nuestra API para que esté disponible en Internet. Lo primero es crearnos una cuenta gratuita en **https://www.pythonanywhere.com**.

$$$$

![pythonanywhere](images/pythonanywhere.png)

$$$$

A continuación, crearemos una carpeta para el proyecto que va a contener el archivo csv con los datos y el archivo `main.py` con el código de nuestra API. Nos vamos a la pestaña de archivos:

![pythonanywhere_files](images/pythonanywhere_files.png)

$$$$

![pythonanywhere_directorios](images/pythonanywhere_directorios.png)


La carpeta la llamaremos API, y tendrá ésta estructura:

```
API
|   main.py
└───data
    │   apple_store.csv
```

#### Archivo `main.py`

Requiere algún cambio:

```python

from flask import Flask, request      # se importan librerias
import pandas as pd
import os


app=Flask(__name__)         # inicializar la aplicacion 


data=None                   # inicializa variables


PATH=os.path.dirname(os.path.abspath(__file__)) # ruta absoluta a este archivo, necesario en pythonanywhere



@app.before_first_request   # antes del primer request
def startup():
    global data   # variable global data
    
    data=pd.read_csv(PATH + '/data/apple_store.csv')   # se sobreescribe la variable global

    
@app.route('/')             
def main():          
    # funcion en la raiz, solo devuelve una string
    return 'Apple Store API'



@app.route('/calc/')  # ruta url a otra parte de la pagina
def calcular():
    # esta funcion devuelve la descripcion del dataframe
    return str(data.describe().T.to_json())




@app.route('/filtrar/')  # ruta url a otra parte de la pagina
def filtrar():
    # funcion para filtar por genero
    
    param=request.args.get('gen')
    
    filtrado=data[data.prime_genre==param].to_json()
    
    return str(filtrado)



@app.route('/selec/')  # ruta de la url
def seleccionar():
    # funcion que selecciona parte del dataframe
    
    # parametros, columnas a seleccionar
    params=request.args   # diccionario, key=nombre del parámetro, value=valor aportado en la url
    
    cols=[v for v in params.values()]  # valores, en este caso nombres de columna
   
    seleccion=data[cols].to_json()
    
    return str(seleccion)
```

#### Finalización

Una vez que hemos subido los archivos, tenemos que establecer la ruta al código fuente y el servidor `wsgi`:

$$$$

![pythonanywhere_source_code](images/pythonanywhere_source_code.png)

$$$$

![pythonanywhere_wsgi](images/pythonanywhere_wsgi.png)


$$$$

Por último debemos **recargar el servidor**:


$$$$

![pythonanywhere_reload](images/pythonanywhere_reload.png)


$$$$

Si surge un error en el servidor, recurriremos al **log**:

$$$$

![pythonanywhere_log](images/pythonanywhere_log.png)

**Prueba API:**

+ Página principal: http://yon.pythonanywhere.com/

+ Cálculo de la descripción: http://yon.pythonanywhere.com/calc/

+ Filtrado por género: http://yon.pythonanywhere.com/filtrar/?gen=Games

+ Selección de columnas: http://yon.pythonanywhere.com/selec/?c1=user_rating&c2=user_rating_ver

$$$$


### Heroku (alternativa a pythonanywhere)

![heroku](images/heroku.webp)

$$$$ El manejo de [Heroku](https://www.heroku.com/) es ligeramente más complejo:


En primer lugar se crea un entorno virtual con el nombre de la aplicacion.

```
python3 -m virtualenv nombre
```

$$$$

Se activa el entorno virtual y se entra en la carpeta...

```
source ~/nombre/bin/activate

cd nombre
```

$$$$


Se inicializa el repo de git...

```
git init
```

$$$$

Se instalan las dependencias...

```
pip install flask gunicorn ....
```

$$$$

Se crea el archivo de requerimientos...

```
pip freeze>requirements.txt 
```

$$$$


Se crea el archivo Procfile...

```
touch Procfile
```

$$$$


y dentro de él lo siguiente:

```
web: gunicorn wsgi:app
```

$$$$

Se crea el archivo runtime.txt..

```
touch runtime.txt
```

$$$$


y dentro de él la versión de python:

```
python-3.7.5
```

$$$$

Se crea el archivo wsgi.py

```
touch wsgi.py
```

$$$$

con el siguiente codigo:
```python
from app.main import app 
  
if __name__ == "__main__": 
        app.run() 

```
$$$$

Una vez realizado este proceso,  se crea la carpeta app, y dentro de ella se ponen todos los archivos de la aplicacion, main.py y todas las demas carpetas.


Una vez hecho esto:

``` 
git add .
git commit -m 'deploy'

heroku login
heroku create nombre-app
```

$$$$

Despues puede dar un fallo, hay que darle el buildpack de python en heroku.

Por último:

```
git push heroku master
```